In [1]:
#!kill -9 $(lsof -t -i:5000) 2>/dev/null
#!kill -9 $(lsof -t -i:5050) 2>/dev/null
#!pkill -f flask 2>/dev/null || true
#!pkill -f ngrok 2>/dev/null || true
#print("Cleared old Flask and ngrok processes.")

!pip install -q flask flask-cors pyngrok transformers accelerate torch rapidfuzz spacy spacy-llm beautifulsoup4 requests

import torch, json, re, gc, time
import requests
from bs4 import BeautifulSoup
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from threading import Thread
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
from rapidfuzz import fuzz, process
import spacy

actor_lists = {
    "EIT Organizations": [
        "European Institute of Innovation and Technology (EIT)",
        "EIT Food", "EIT Health", "EIT InnoEnergy", "EIT Digital",
        "EIT RawMaterials", "EIT Manufacturing", "EIT Urban Mobility",
        "EIT Climate-KIC", "EIT Culture & Creativity"
    ],
    "Universities / Research Institutes": [
        "KU Leuven", "KTH Royal Institute of Technology", "Universitat Politècnica de Catalunya",
        "Eindhoven University of Technology (TU/e)", "Instituto Superior Técnico",
        "ESADE Business School", "Politecnico di Torino", "École Polytechnique",
        "Aalto University", "Université Paris Sciences et Lettres (PSL)", "Université Paris-Saclay",
        "Grenoble INP Institute of Technology", "École des Ponts ParisTech (ENPC)",
        "AGH University of Science and Technology", "Imperial College London",
        "University of Oxford", "University of Cambridge", "IESE Business School",
        "IMIM – Hospital del Mar Medical Research Institute",
        "Delft University of Technology", "University of Porto",
        "University of Debrecen", "University of Luxembourg",
        "Leitat Technology Center", "Fraunhofer Institute",
        "Technical University of Munich", "RWTH Aachen University",
        "Czech Technical University in Prague",
        "Uppsala University", "Tartu University Hospital",
        "North Lisbon University Hospital Centre", "Hospital Clínic",
        "University of Maastricht", "Eötvös Lorand University", "KTH Institute",
        "University of Pécs"
    ],
    "Companies / Corporates": [
        "Schneider Electric", "Siemens Healthineers", "Roche", "Bayer", "Sanofi",
        "Johnson & Johnson", "IBM", "Philips", "Bosch", "Microsoft", "GE Healthcare",
        "URGO Group", "Ferrer", "Matmut", "ABB", "ENGIE", "Airbus", "ArcelorMittal",
        "Veolia", "TotalEnergies", "Nestlé", "Danone", "PepsiCo", "Unilever",
        "Shell", "Vattenfall", "Iberdrola", "Enel", "Equinor", "Nokia", "Ericsson"
    ],
    "Startups / SMEs": [
        "iLoF", "Sleepiz", "Optellum", "Idoven", "PIPRA", "Antegenes",
        "Clinomic", "Unhindr", "Leuko", "Ochre Bio",
        "Hearts Radiant", "Allelica", "SolasCure", "Peptomyc",
        "Oxford Endovascular", "Tubulis", "SideROS", "Emperra",
        "FasTeesH", "MEDIKURA", "SpinDiag", "Selio Medical",
        "Damibu", "Telomium", "Tracegrow", "Entremo",
        "Recycleye", "InnoTractor", "LMAD", "OvaExpert",
        "Feno", "Ganymed Robotics",
        "ABLE Human Motion", "FLOWTION", "SeizeIT", "NanoRacks",
        "AMEN New Technologies", "CroíValve"
    ],
    "Government / Public Sector": [
        "European Commission", "European Union", "European Parliament",
        "European Investment Fund (EIF)", "European Investment Bank (EIB)",
        "National Health Service (UK)", "Ministry of Human Capacities (Hungary)",
        "German Federal Ministry of Education and Research", "City of Debrecen",
        "Spanish Ministry of Science and Innovation",
        "French Ministry of Higher Education and Research",
        "Italian Ministry for Economic Development",
        "Polish Ministry of Climate and Environment",
        "European Council", "European Court of Auditors"
    ],
    "Networks / Consortia / Foundations": [
        "European Battery Alliance", "European Youth Energy Network",
        "CommUnity+", "Foundation for Management and Industrial Research (MIR)",
        "Enterprise Europe Network", "HealthTech For Care",
        "Venture Centre of Excellence (VCoE)",
        "BioMed Alliance", "WE Health Consortium", "CLOSE Consortium",
        "InnoEnergy Alliance", "RawMaterials Academy", "Urban Mobility Academy",
        "Climate-KIC Alumni Association", "EIT Digital Alumni", "RIS Hub Network",
        "Regional Innovation Scheme (RIS)", "EIT Community Booster",
        "Supernovas Programme", "Fondation de l'Avenir",
        "Polish Medical Mission", "BRIGHT Project Innovation Team"
    ],
    "Investors / Funding Bodies": [
        "Santander InnoEnergy Climate Fund", "EBA Strategic Battery Materials Fund",
        "Aescuvest Crowdfunding Platform", "Zafir Capital", "Alta Life Sciences",
        "YES!Delft", "Startup Wise Guys", "Speedinvest", "Seedcamp",
        "EASME – Executive Agency for SMEs", "Horizon Europe",
        "European Innovation Council (EIC)", "Business Finland",
        "Vinnova", "CDTI Spain", "Bpifrance"
    ]
}

ALL_ACTORS = []
for category_actors in actor_lists.values():
    ALL_ACTORS.extend(category_actors)

ACTOR_LOWER_MAP = {actor.lower(): actor for actor in ALL_ACTORS}

print(f"Loaded {len(ALL_ACTORS)} actors\n")


# LOAD SPACY
print("Loading SpaCy English model...")
try:
    nlp_spacy = spacy.load("en_core_web_sm")
    print("✓ SpaCy loaded successfully")
except:
    print("Downloading SpaCy English model...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp_spacy = spacy.load("en_core_web_sm")
    print("✓ SpaCy downloaded and loaded")

# LOAD MISTRAL
print("\nLoading Mistral 7B...")
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)
print("Mistral loaded")

# LOAD GEMMA
print("\nLoading Gemma 7B...")
hf_token = userdata.get("HF_TOKEN2")
gemma_tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-7b-it",
    token=hf_token
)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
    token=hf_token
)
print("Gemma loaded\n")

# HELPER FUNCTIONS
def fuzzy_match_actor(name, threshold=60):
    if not name or len(name) < 2:
        return None

    name_lower = name.lower().strip()

    if name_lower in ACTOR_LOWER_MAP:
        return ACTOR_LOWER_MAP[name_lower]

    result = process.extractOne(
        name,
        ALL_ACTORS,
        scorer=fuzz.token_sort_ratio,
        score_cutoff=threshold
    )

    if result:
        matched_actor, score, _ = result
        return matched_actor

    return None

def validate_triplet(triplet):
    role_raw = triplet.get("role", "")
    counterrole_raw = triplet.get("counterrole", "")
    practice = triplet.get("practice", "")

    if not role_raw or not counterrole_raw or not practice:
        return None, "Missing fields"

    role_raw = role_raw.strip()
    counterrole_raw = counterrole_raw.strip()
    practice = practice.strip()

    if len(role_raw) < 2 or len(counterrole_raw) < 3:
        return None, "Too short"

    counterrole_lower = counterrole_raw.lower()
    generic_terms = [
        "us", "we", "you", "they", "them", "it", "this", "that",
        "people", "businesses", "companies", "organizations", "partners",
        "stakeholders", "members", "community", "projects", "programmes",
        "opportunities", "solutions", "services", "products",
        "Europe", "innovation", "development", "healthcare", "health"
    ]

    if counterrole_lower in generic_terms:
        return None, "Generic counterrole"

    if len(counterrole_raw) > 100:
        return None, "Too long"

    vague_practices = ["has", "is", "does", "makes", "gets", "uses"]
    if practice.lower() in vague_practices:
        return None, "Vague practice"

    role_matched = fuzzy_match_actor(role_raw, threshold=60)

    if not role_matched:
        return None, "Role not in list"

    triplet["role"] = role_matched
    triplet["counterrole"] = counterrole_raw

    return triplet, "valid"

def chunk_text(text, chunk_size=1500):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks, current = [], ""

    for sent in sentences:
        if len(current) + len(sent) <= chunk_size:
            current += sent + " "
        else:
            if current.strip():
                chunks.append(current.strip())
            current = sent + " "

    if current.strip():
        chunks.append(current.strip())

    return chunks

def make_base_prompt(text_chunk, user_prompt=""):
    actor_examples = "\n".join([f"  • {actor}" for actor in ALL_ACTORS[:30]])

    base_prompt = f"""Extract organizational relationships as JSON.

CONSTRAINT: 'role' MUST be from this list:
{actor_examples}
  • ... and {len(ALL_ACTORS)-30} more actors

RULES:
1. role: Organization taking action (from list above)
2. practice: Specific action verb (e.g., "fund", "partner with", "support")
3. counterrole: Specific named entity (not generic terms)
4. context: Exact sentence

Output ONLY valid JSON array."""

    if user_prompt and user_prompt.strip():
        base_prompt += f"\n\nADDITIONAL INSTRUCTIONS:\n{user_prompt.strip()}"

    base_prompt += f"\n\nTEXT:\n{text_chunk}\n\nJSON:"

    return base_prompt

def make_gemma_prompt(text_chunk, user_prompt=""):
    base = make_base_prompt(text_chunk, user_prompt)
    return f"<start_of_turn>user\n{base}<end_of_turn>\n<start_of_turn>model\n"

# EXTRACTION FUNCTIONS
def extract_with_llm(text, user_prompt, model_name, tokenizer, model):
    if model_name == "gemma":
        prompt = make_gemma_prompt(text, user_prompt)
    else:
        prompt = make_base_prompt(text, user_prompt)

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3000).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=700,
                temperature = 0.5 if model_name == "gemma" else 0.3,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True,
                output_scores=True
            )

        result = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        result = result.replace(prompt, "").strip()

        scores = outputs.scores
        token_confidences = [torch.max(torch.nn.functional.softmax(score[0], dim=-1)).item() for score in scores]
        avg_confidence = sum(token_confidences) / len(token_confidences) if token_confidences else 0.5

        clean_output = result

        if "```json" in clean_output:
            clean_output = clean_output.split("```json")[1].split("```")[0].strip()
        elif "```" in clean_output:
            clean_output = clean_output.split("```")[1].split("```")[0].strip()

        for prefix in ["JSON:", "OUTPUT:"]:
            if clean_output.upper().startswith(prefix):
                clean_output = clean_output[len(prefix):].strip()

        start, end = clean_output.find("["), clean_output.rfind("]")
        if start == -1:
            start, end = clean_output.find("{"), clean_output.rfind("}")
            if start != -1 and end != -1:
                clean_output = "[" + clean_output[start:end+1] + "]"

        json_text = clean_output[start:end+1] if start != -1 and end != -1 else "[]"

        triples = json.loads(json_text)
        if isinstance(triples, dict):
            triples = [triples]

        validated_triples = []
        for triple in triples:
            triple['model_confidence'] = round(avg_confidence, 3)
            triple.setdefault('role', '')
            triple.setdefault('practice', '')
            triple.setdefault('counterrole', '')
            triple.setdefault('context', '')

            validated, reason = validate_triplet(triple)

            if validated:
                validated_triples.append(validated)

        return validated_triples

    except Exception as e:
        return []
    finally:
        torch.cuda.empty_cache()
        gc.collect()

def extract_with_spacy_llm(text, user_prompt):
    """
    TRUE SpaCy-LLM Hybrid:
    1. SpaCy does NLP preprocessing (tokenization, NER, POS tagging)
    2. Mistral does relation extraction with entity context
    """
    try:
        doc = nlp_spacy(text[:5000])  # Limit to 5000 chars for speed

        # Extract named entities
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        potential_roles = []
        text_lower = text.lower()
        for actor in ALL_ACTORS:
            if actor.lower() in text_lower:
                potential_roles.append(actor)

        if not potential_roles:
            potential_roles = ["EIT Health", "EIT Food", "EIT InnoEnergy"]

        # Extract verbs that might be practices
        verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"][:20]

        spacy_enhanced_prompt = f"""Extract organizational relationships using NLP analysis:

DETECTED ORGANIZATIONS: {', '.join(potential_roles[:10])}
NAMED ENTITIES: {', '.join([f"{e[0]} ({e[1]})" for e in entities[:15]])}
KEY VERBS: {', '.join(verbs[:15])}

Task: Extract triplets (role → practice → counterrole)
- role: MUST be from detected organizations above
- practice: Action verb (preferably from key verbs)
- counterrole: Specific named entity (from detected entities or text)
- context: Exact sentence from text

STRICT RULES:
1. role must match detected organizations
2. counterrole must be specific (NOT generic terms like "partners", "stakeholders")
3. practice must be a clear action

Text to analyze:
{text[:2000]}

Output ONLY JSON array:
[{{"role": "...", "practice": "...", "counterrole": "...", "context": "..."}}]

JSON:"""

        # Mistral for relation extraction with SpaCy context
        inputs = mistral_tokenizer(
            spacy_enhanced_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=3000
        ).to(mistral_model.device)

        with torch.no_grad():
            outputs = mistral_model.generate(
                **inputs,
                max_new_tokens=700,
                temperature=0.3,
                do_sample=True,
                pad_token_id=mistral_tokenizer.eos_token_id,
                return_dict_in_generate=True,
                output_scores=True
            )

        result = mistral_tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        result = result.replace(spacy_enhanced_prompt, "").strip()

        scores = outputs.scores
        token_confidences = [
            torch.max(torch.nn.functional.softmax(score[0], dim=-1)).item()
            for score in scores
        ]
        avg_confidence = sum(token_confidences) / len(token_confidences) if token_confidences else 0.75

        # Clean JSON output
        clean_output = result
        if "```json" in clean_output:
            clean_output = clean_output.split("```json")[1].split("```")[0].strip()
        elif "```" in clean_output:
            clean_output = clean_output.split("```")[1].split("```")[0].strip()

        for prefix in ["JSON:", "OUTPUT:"]:
            if clean_output.upper().startswith(prefix):
                clean_output = clean_output[len(prefix):].strip()

        start, end = clean_output.find("["), clean_output.rfind("]")
        if start == -1:
            start, end = clean_output.find("{"), clean_output.rfind("}")
            if start != -1 and end != -1:
                clean_output = "[" + clean_output[start:end+1] + "]"

        json_text = clean_output[start:end+1] if start != -1 and end != -1 else "[]"

        relations = json.loads(json_text)
        if isinstance(relations, dict):
            relations = [relations]

        validated_triples = []
        for rel in relations:
            triplet = {
                "role": rel.get("role", ""),
                "practice": rel.get("practice", ""),
                "counterrole": rel.get("counterrole", ""),
                "context": rel.get("context", ""),
                "model_confidence": round(avg_confidence, 3)
            }

            validated, reason = validate_triplet(triplet)
            if validated:
                validated_triples.append(validated)

        return validated_triples

    except Exception as e:
        print(f"SpaCy-LLM error: {e}")
        return []
    finally:
        torch.cuda.empty_cache()
        gc.collect()

# FLASK APP
app = Flask(__name__)
CORS(app)

@app.route("/extract_triplets", methods=["POST"])
def extract_endpoint():
    data = request.get_json(force=True)
    text = data.get("text", "")
    model_choice = data.get("model", "Mistral 7B").strip()
    user_prompt = data.get("user_prompt", "")
    max_triplets = data.get("max_triplets", None)

    if not text.strip():
        return jsonify({"error": "No text provided"}), 400

    model_map = {
        "Mistral 7B": "mistral",
        "Gemma 7B": "gemma",
        "SpacyLLM": "spacy-llm"
    }

    model_key = model_map.get(model_choice, "mistral")

    model_names = {
        "mistral": "Mistral 7B",
        "gemma": "Gemma 7B",
        "spacy-llm": "SpaCy-LLM (SpaCy NER + Mistral Extraction)"
    }

    print(f"\n{'='*70}")
    print(f"CONFIGURATION:")
    print(f"  Received: {len(text):,} characters")
    print(f"  Model selected: {model_names[model_key]}")
    print(f"  Model key: {model_key}")
    print(f"  User prompt: {'Yes' if user_prompt else 'No'}")
    print(f"  Max triplets: {max_triplets if max_triplets else 'Unlimited'}")
    print(f"{'='*70}")

    chunks = chunk_text(text, chunk_size=1500)
    total_chunks = len(chunks)

    print(f"Total chunks: {total_chunks}")
    print(f"Estimated time: {total_chunks*2/60:.1f} minutes")
    print(f"{'='*70}\n")

    all_triplets = []

    health_end = 111637
    food_end = health_end + 122735

    for i, chunk in enumerate(chunks):
        if max_triplets and len(all_triplets) >= max_triplets:
            print(f"\nReached max triplets limit: {max_triplets}")
            print(f"Stopping at chunk {i+1}/{total_chunks}")
            break

        char_pos = i * 1500
        if char_pos < health_end:
            current_org = "EIT Health"
        elif char_pos < food_end:
            current_org = "EIT Food"
        else:
            current_org = "EIT InnoEnergy"

        print(f"\n{'='*70}")
        print(f"CHUNK {i+1}/{total_chunks} | {current_org}")
        print(f"MODEL: {model_names[model_key]}")
        print(f"{'='*70}")

        try:
            if model_key == "spacy-llm":
                triples = extract_with_spacy_llm(chunk, user_prompt)
            elif model_key == "gemma":
                triples = extract_with_llm(chunk, user_prompt, "gemma", gemma_tokenizer, gemma_model)
            else:
                triples = extract_with_llm(chunk, user_prompt, "mistral", mistral_tokenizer, mistral_model)

            if triples:
                for triple in triples:
                    if max_triplets and len(all_triplets) >= max_triplets:
                        print(f"\nReached limit during chunk processing")
                        break

                    all_triplets.append(triple)

                    role = triple.get('role', 'Unknown')
                    practice = triple.get('practice', 'Unknown')
                    counterrole = triple.get('counterrole', 'Unknown')

                    print(f"{role} -> {practice} -> {counterrole}")

                if max_triplets and len(all_triplets) >= max_triplets:
                    print(f"\nStopping: reached {max_triplets} triplets")
                    break

                print(f"\nTotal so far: {len(all_triplets)} triplets")
            else:
                print(f"No valid triplets")
                print(f"Total so far: {len(all_triplets)} triplets")

            time.sleep(0.3)

        except Exception as e:
            print(f"Error: {str(e)[:100]}")
            continue

    print(f"\n{'='*70}")
    print(f"EXTRACTION COMPLETE with {model_names[model_key]}!")
    print(f"Total triplets: {len(all_triplets)}")
    print(f"{'='*70}\n")

    formatted_triplets = []
    for idx, triple in enumerate(all_triplets):
        formatted_triplets.append({
            "id": idx + 1,
            "text": triple.get("context", "No context"),
            "community": "EIT Community",
            "extracted": {
                "role": triple.get("role", "Unknown"),
                "practice": triple.get("practice", "Unknown"),
                "counterrole": triple.get("counterrole", "Unknown")
            },
            "confidence": triple.get("model_confidence", 0.5),
            "validated": None
        })

    return jsonify({
        "total_chunks": total_chunks,
        "total_triplets": len(formatted_triplets),
        "triplets": formatted_triplets,
        "model_used": model_names[model_key],
        "status": "success"
    })

# ADDED SCRAPE ENDPOINT
@app.route("/scrape_url", methods=["POST"])
def scrape_endpoint():
    data = request.get_json(force=True)
    url = data.get("url", "")

    if not url:
        return jsonify({"error": "No URL provided"}), 400

    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract text from paragraphs
        paragraphs = [p.get_text().strip() for p in soup.find_all('p') if p.get_text().strip()]
        cleaned_text = "\n\n".join(paragraphs)

        return jsonify({"text": cleaned_text, "source": url})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/models", methods=["GET"])
def get_models():
    return jsonify({
        "models": [
            {"id": "Mistral 7B", "name": "Mistral 7B", "type": "LLM"},
            {"id": "Gemma 7B", "name": "Gemma 7B", "type": "LLM"},
            {"id": "SpacyLLM", "name": "SpaCy-LLM Hybrid", "type": "Hybrid"}
        ]
    })

# START SERVER
print("\nStarting server...\n")

authtoken = userdata.get("NGROK")
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(5050)

print("="*70)
print(f"Models: {public_url.public_url}/models")
print(f"Extract: {public_url.public_url}/extract_triplets")
print(f"Scrape: {public_url.public_url}/scrape_url") # <--- Updated print
print("="*70)
print("\nAvailable models:")
print(" Mistral 7B")
print(" Gemma 7B")
print(" SpaCy-LLM")
print("\nReady!\n")
print("="*70)

Thread(target=lambda: app.run(port=5050, debug=False, use_reloader=False)).start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nStopped.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 25.4 MB/s eta 0:00:00
Loaded 168 actors

Loading SpaCy English model...
✓ SpaCy loaded successfully

Loading Mistral 7B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Gemma loaded


Starting server...

Models: https://nonfelicitous-reasonedly-cadence.ngrok-free.dev/models
Extract: https://nonfelicitous-reasonedly-cadence.ngrok-free.dev/extract_triplets
Scrape: https://nonfelicitous-reasonedly-cadence.ngrok-free.dev/scrape_url

Available models:
 Mistral 7B
 Gemma 7B
 SpaCy-LLM

Ready!

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5050
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Nov/2025 04:21:33] "OPTIONS /scrape_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2025 04:21:35] "POST /scrape_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2025 04:22:29] "OPTIONS /extract_triplets HTTP/1.1" 200 -



CONFIGURATION:
  Received: 187,381 characters
  Model selected: SpaCy-LLM (SpaCy NER + Mistral Extraction)
  Model key: spacy-llm
  User prompt: Yes
  Max triplets: 10
Total chunks: 141
Estimated time: 4.7 minutes


CHUNK 1/141 | EIT Health
MODEL: SpaCy-LLM (SpaCy NER + Mistral Extraction)
EIT Health -> bring -> brightest minds
EIT Health -> connect -> vast community
EIT Health -> enable -> people in Europe

Total so far: 3 triplets

CHUNK 2/141 | EIT Health
MODEL: SpaCy-LLM (SpaCy NER + Mistral Extraction)
No valid triplets
Total so far: 3 triplets

CHUNK 3/141 | EIT Health
MODEL: SpaCy-LLM (SpaCy NER + Mistral Extraction)


INFO:werkzeug:127.0.0.1 - - [22/Nov/2025 04:23:29] "POST /extract_triplets HTTP/1.1" 200 -


EIT Health -> engages -> healthcare thought-leaders
EIT Health -> supports -> 2998 + start-ups and scale-ups
EIT Health -> trains -> 65k + students and professionals
EIT Health -> launches -> 123 products and services
EIT Health -> brings -> cutting-edge technologies
EIT Health -> nominates -> three EIT Award nominees
Hospital Clínic -> reinforces -> patient participation

Stopping: reached 10 triplets

EXTRACTION COMPLETE with SpaCy-LLM (SpaCy NER + Mistral Extraction)!
Total triplets: 10


Stopped.
